In [57]:
!pip install datasets evaluate transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [58]:
import os

# Data processing
import pandas as pd
import numpy as np

# Modeling
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback

# Hugging Face Dataset
from datasets import Dataset

# Model performance evaluation
import evaluate

In [59]:
df = pd.read_csv("data/amazon.csv")

for root, _, files in os.walk("data"):
    for filename in files:
        temp_df = pd.read_csv(os.path.join(root, filename))
        df = pd.concat([df, temp_df], axis=0, sort=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18447 entries, 0 to 668
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          18447 non-null  int64  
 1   Дата                10873 non-null  object 
 2   Время               10873 non-null  object 
 3   Цена до             10873 non-null  float64
 4   Цена после          10873 non-null  float64
 5   Разница в долларах  10873 non-null  float64
 6   Дельта в процентах  10873 non-null  float64
 7   Текст новости       10873 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 1.3+ MB


In [60]:
df = df.drop_duplicates(subset=["Текст новости"], keep=False)
df.drop(columns=[df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True)
df.rename(columns={"Цена до": "price_before", "Цена после": "price_after", "Дата": "date", "Время": "Time", "Текст новости": "news_text"}, inplace=True)

df["absolute_price_difference"] = df["price_after"] - df["price_before"]
df["percentage_price_difference"] = df["absolute_price_difference"] / df["price_before"] * 100
df["label"] = np.where(df["absolute_price_difference"] > 0, 1, 0)
df = df[["news_text", "label"]].copy()
df.head()

,news_text,label
0,Check Out What Whales Are Doing With NOC\n,0
1,What 7 Analyst Ratings Have To Say About North...,0
2,7 Analysts Have This to Say About Northrop Gru...,0
3,"Benzinga's Top Ratings Upgrades, Downgrades Fo...",0
5,Looking Into Northrop Grumman's Recent Short I...,1


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3593 entries, 0 to 667
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   news_text  3593 non-null   object
 1   label      3593 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 84.2+ KB


In [62]:
train_data = df.sample(frac=0.8, random_state=42)

# Testing dataset
test_data = df.drop(train_data.index)

# Check the number of records in training and testing dataset.
print(f'The training dataset has {len(train_data)} records.')
print(f'The testing dataset has {len(test_data)} records.')

The training dataset has 2874 records.
The testing dataset has 54 records.


In [63]:
train_data["news_text"].tolist()

["Will Chart Analysts Notice Bad Omen on Netflix's Chart\n",
 "Tuesday's Market Minute: Goodbye, January\n",
 'Mid-Afternoon Market Update: Nasdaq Tumbles 300 Points; Neuronetics Shares Spike Higher\n',
 'Why This Pfizer Analyst Is Sidelined After Beat-And-Raise Q1\n',
 "Benzinga's Top Upgrades, Downgrades For October 28, 2020\n",
 'YouTube To Delete Videos On Vaccine Misinformation\n',
 'CEO Eyes Breakthrough Year as DroneUp Ramps Up Walmart Delivery Pilot\n',
 'RTFKT, Owned By Nike, Permits Commercial Rights To NFTs For CloneX Holders\n',
 'Mid-Morning Market Update: Markets Open Lower; Pfizer To Acquire Arena Pharmaceuticals\n',
 "Health Canada Approves Moderna's COVID-19 Vaccine For Kids Aged 6-11 Years\n",
 "60 Stocks Moving In Friday's Mid-Day Session\n",
 "Online Backlash Percolates Over Elon Musk's 'SNL' Hosting Gig\n",
 '300M Dogecoin Moved In One Go As Market Enthusiasm Sends Whales Into Frenzy\n',
 'Is Apple Headed To $173 Next? What To Expect As The Stock Breaks Bullishly F

In [64]:
hg_train_data = Dataset.from_pandas(train_data)
hg_test_data = Dataset.from_pandas(test_data)

In [65]:
print(f'The length of hg_train_data is {len(hg_train_data)}.\n')

# Check one review
hg_train_data[0]

The length of hg_train_data is 2874.



{'news_text': "Will Chart Analysts Notice Bad Omen on Netflix's Chart\n",
 'label': 0,
 '__index_level_0__': 426}

In [66]:
# Tokenizer from a pretrained model
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# Take a look at the tokenizer
tokenizer

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/alexa/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}

loading file https://huggingface.co/bert-base-cased/resolve/ma

PreTrainedTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [67]:
# Mapping between special tokens and their IDs.
print(f'The unknown token is {tokenizer.unk_token} and the ID for the unkown token is {tokenizer.unk_token_id}.')
print(f'The seperator token is {tokenizer.sep_token} and the ID for the seperator token is {tokenizer.sep_token_id}.')
print(f'The pad token is {tokenizer.pad_token} and the ID for the pad token is {tokenizer.pad_token_id}.')
print(f'The sentence level classification token is {tokenizer.cls_token} and the ID for the classification token is {tokenizer.cls_token_id}.')
print(f'The mask token is {tokenizer.mask_token} and the ID for the mask token is {tokenizer.mask_token_id}.')

The unknown token is [UNK] and the ID for the unkown token is 100.
The seperator token is [SEP] and the ID for the seperator token is 102.
The pad token is [PAD] and the ID for the pad token is 0.
The sentence level classification token is [CLS] and the ID for the classification token is 101.
The mask token is [MASK] and the ID for the mask token is 103.


In [68]:
# Funtion to tokenize data
def tokenize_dataset(data):
    return tokenizer(data["news_text"], 
                     max_length=32,
                     truncation=True, 
                     padding="max_length")

# Tokenize the dataset
dataset_train = hg_train_data.map(tokenize_dataset)
dataset_test = hg_test_data.map(tokenize_dataset)

Map:   0%|          | 0/2874 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

In [69]:
# Take a look at the data
print(dataset_train)
print(dataset_test)

Dataset({
    features: ['news_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2874
})
Dataset({
    features: ['news_text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 54
})


In [70]:

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/alexa/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file https://huggingface.co/bert-base-unca

In [92]:
training_args = TrainingArguments(
    output_dir="./sentiment_transfer_learning_transformer/",          
    logging_dir='./sentiment_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=20,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [72]:
print(f'There are {len(evaluate.list_evaluation_modules())} evaluation models in Hugging Face.\n')

# List all evaluation metrics
evaluate.list_evaluation_modules()

There are 157 evaluation models in Hugging Face.



['lvwerra/test',
 'precision',
 'code_eval',
 'roc_auc',
 'cuad',
 'xnli',
 'rouge',
 'pearsonr',
 'mse',
 'super_glue',
 'comet',
 'cer',
 'sacrebleu',
 'mahalanobis',
 'wer',
 'competition_math',
 'f1',
 'recall',
 'coval',
 'mauve',
 'xtreme_s',
 'bleurt',
 'ter',
 'accuracy',
 'exact_match',
 'indic_glue',
 'spearmanr',
 'mae',
 'squad',
 'chrf',
 'glue',
 'perplexity',
 'mean_iou',
 'squad_v2',
 'meteor',
 'bleu',
 'wiki_split',
 'sari',
 'frugalscore',
 'google_bleu',
 'bertscore',
 'matthews_correlation',
 'seqeval',
 'trec_eval',
 'rl_reliability',
 'jordyvl/ece',
 'angelina-wang/directional_bias_amplification',
 'cpllab/syntaxgym',
 'lvwerra/bary_score',
 'kaggle/amex',
 'kaggle/ai4code',
 'hack/test_metric',
 'yzha/ctc_eval',
 'codeparrot/apps_metric',
 'mfumanelli/geometric_mean',
 'daiyizheng/valid',
 'poseval',
 'erntkn/dice_coefficient',
 'mgfrantz/roc_auc_macro',
 'Vlasta/pr_auc',
 'gorkaartola/metric_for_tp_fp_samples',
 'idsedykh/metric',
 'idsedykh/codebleu2',
 'idsed

In [73]:
# Function to compute the metric
def compute_metrics(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)
    return metric.compute(predictions=predictions, references=labels)

In [93]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, news_text. If __index_level_0__, news_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/alexa/miniconda3/envs/tf/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 2874
  Num Epochs = 20
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 14380


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, news_text. If __index_level_0__, news_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54
  Batch size = 4
Saving model checkpoint to ./sentiment_transfer_learning_transformer/checkpoint-719
Configuration saved in ./sentiment_transfer_learning_transformer/checkpoint-719/config.json
Model weights saved in ./sentiment_transfer_learning_transformer/checkpoint-719/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, news_text. If __index_level_0__, news_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples 

TrainOutput(global_step=1438, training_loss=0.25818890822281926, metrics={'train_runtime': 147.8863, 'train_samples_per_second': 388.677, 'train_steps_per_second': 97.237, 'total_flos': 94522646638080.0, 'train_loss': 0.25818890822281926, 'epoch': 2.0})

In [94]:
y_test_predict = trainer.predict(dataset_test)

# Take a look at the predictions
y_test_predict

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, news_text. If __index_level_0__, news_text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 54
  Batch size = 4


PredictionOutput(predictions=array([[-3.3516743 ,  3.8316574 ],
       [-3.8557315 ,  4.489566  ],
       [-2.242092  ,  2.7173576 ],
       [ 2.4116144 , -2.6358116 ],
       [ 3.076421  , -4.103788  ],
       [ 3.3855612 , -4.204203  ],
       [ 3.5259123 , -4.2156367 ],
       [-3.8095613 ,  4.4856167 ],
       [-2.8397956 ,  3.1818435 ],
       [ 2.989258  , -3.9587328 ],
       [-0.20726822,  0.13903147],
       [-3.822425  ,  4.5614123 ],
       [ 3.8006265 , -4.498038  ],
       [ 3.3516002 , -4.081126  ],
       [ 2.636251  , -2.8871162 ],
       [ 3.3738399 , -3.9633384 ],
       [ 1.7367955 , -2.3856862 ],
       [-1.159985  ,  1.654037  ],
       [-3.4666836 ,  3.9035947 ],
       [-3.4884396 ,  4.0015063 ],
       [ 3.2989392 , -4.065156  ],
       [-2.9920397 ,  3.746995  ],
       [ 2.432497  , -3.4390755 ],
       [ 3.3325574 , -4.117629  ],
       [-3.3811824 ,  3.8985913 ],
       [-1.359944  ,  1.6617783 ],
       [-3.017413  ,  3.533588  ],
       [ 3.199379  , -3.89